In [27]:
from bs4 import BeautifulSoup
import csv
import json
from datetime import datetime
import re
import sys, os
import emoji
import time
import os
import ast


In [28]:
#stole from Richa's code, remove emojis from text
def deEmojify(text):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                    "]+", re.UNICODE)
    return re.sub(emoj, '', text)

def remove_extras(x: str):
    '''
    This function removes the extras character leftover from
    the html parser.
    
    Args:
        x (str): any string.
    Returns:
        str: corrected string.
    '''
    #remove html tidbits 
    #remove emojis
    x = deEmojify(x)
    x = emoji.replace_emoji(x, replace='')
    x = x.replace("xad", "")
    x = x.replace("u200b", "")
    x = x.replace("'", "")
    x = x.replace("}}]", "")
    x = x.replace("}}", "")
    x = x.replace("\\t", " ")
    x = x.replace(" | ", ". ")
    x = x.replace('\'', "'")
    x = x.replace(" \'", "'")
    x = x.replace(":\'", "")
    x = x.replace("> ", "")
    x = x.replace("[", "")
    x = x.replace("]", "")
    x = x.replace("**", "")
    x = x.replace("*", "")
    x = x.replace("---", " ")
    x = x.replace("\\n", ". ")
    x = x.replace('\\', "")
    x = x.replace('~~', "")
    x = x.replace("_", "")
    x = x.replace("....", " ")
    x = x.replace("...", ". ")
    x = x.replace("..", ". ")
    x = x.replace(". . ", ". ")
    x = x.replace(".  . ", ". ")
    x = x.replace("^#1", ",").replace("^#2", ",").replace("^#3", ",").replace("^#4", ",").replace("^#5", ",")
    x = x.replace('\n', ' ')
    #remove user handles
    # x = re.sub('@[^\s]+','',x)
    #regex to remove URLs
    x = re.sub(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '', x)
    #remove unk tokens
    # x = re.sub('unk', '', x)
    x = x.replace("()", "")
    #removing anything that's not alphabets
    # x = re.sub('[^A-Z a-z]+', '', x)
    x = x.strip()
    return x

#Check if the corpus actually contains these keywords
def contains_keyword(text): 
    keywords = [
        r'Gaza\w*',
        r'Israel\w*',
        r'West.*Bank|Bank.*West',
        r'Tel.*Aviv',
        r'Tel.*Aviv-Yafo',
        r'Bedouin\w*',
        r'Palestin\w*',
        r'Middle East\w*',
        r'Middle Eastern',
        r'Jew\w*',
        r'rabbi',
        r'Muslim\w*',
        r'Islam\w*',
        r'Jihad',
        r'Antisemiti\w*',
        r'Zionis\w*',
        r'IDF',
        r'IOF',
        r'Hamas',
        r'Massacre',
        r'Genocide',
        r'Ceasefire',
        r'Terroris\w*',
        r'Netanyahu',
        r'Histadrut',
        r'Haniyeh',
        r'Yahya',
        r'Sinwar',
        r'Fatah',
        r'Mohammed'
        r'Deif'
    ]
    for keyword in keywords:
        if re.search(keyword, text, re.IGNORECASE):
            return True
    return False

def extract_usernames(text):
    pattern = r'@\w+'
    usernames = re.findall(pattern, text)
    return usernames

In [29]:
count = 1
number = 4


#After conflict
# until = "2024-09-22"
# since = "2023-10-07"

#Before conflict
until = "2023-10-06"
since = "2022-10-07"

#Creating file paths

raw_data_folder = os.path.join("Raw Data", "Before") #Change to Before and After as needed
parsed_folder = os.path.join("Parsed Data", "Before") #Change to Before and After as needed

# Correct time line 
raw_json = os.path.join(raw_data_folder, f"{number} - US_tweets_{until}_{since}.json")
cleaned_json = os.path.join(raw_data_folder, f"{number} - US_tweets_cleaned_{until}_{since}.json")

# # New Scraped delete later
# raw_json = os.path.join(raw_data_folder, f"{number} - US_tweets_2024-10-07_2023-10-07.json")
# cleaned_json = os.path.join(raw_data_folder, f"{number} - US_tweets_cleaned_2024-10-07_2023-10-07.json")

output_path = os.path.join(parsed_folder, f"{number} - Parsed_{until}_{since}.csv")
output_cleaned_path = os.path.join(parsed_folder, f"{number} - Parsed_{until}_{since}.csv")


In [30]:
all_tweets = []
account_processed = []
account_pre_process = []
account_for_retry = []
account_irrelevant = []
discard = 0

# Restore the built-in list function
import builtins
list = builtins.list

try: 
    with open(raw_json, encoding='utf-8') as f:
        data = json.load(f)


    # Prepare CSV filename
    usernames = list(data.keys())
    # print(len(usernames))

    for u in usernames:
        if u not in account_pre_process:
            account_pre_process.append(u) #List of usernames before getting discarded

    # Prepare CSV file and write headers
    # csv_filename = r'''X\US_parsed.csv''' #PC
    with open(output_path, 'w', newline='', encoding='utf-8') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(['Poster', 'Date', 'Tweet', 'No of Likes', 'No of Retweets', 'No of Replies', 'No of Views'])
        
        for name in usernames: 
            try:
                if len(data[name]) == 0: #Account with no data
                    account_for_retry.append(name)
                    print(f"Retry button appeared: {name}")

                for tweet_html in data[name]: 
                    print(f"Processing user: {name}")
                    all_username = []
                    all_time = []
                    all_likes = []
                    all_comments = []
                    all_retweets = []
                    all_views = []

                    soup = BeautifulSoup(tweet_html, 'html.parser')

                    #Check if there's an error with the html file
                    span_text = "Something went wrong. Try reloading."
                    span = soup.find('span', string=span_text)
                    if span:
                        # Check if the span is within a div
                        parent_div = span.find_parent('div')
                        if parent_div:
                            account_for_retry.append(name)
                            print(f"Span text '{span_text}' found. Skipping to the next account!")
                            continue
                        else:
                            print(f"Span with text '{span_text}' found, but not within a div.")

                    # Extract all tweet texts
                    tweets = soup.find_all(attrs={"data-testid": "tweetText"})
                    tweet_texts = [remove_extras(tweet.get_text(strip=False)) for tweet in tweets]
                    
                    # Select the time value
                    time_element = soup.find_all('time')
                    for t in time_element:
                        datetime_value = t['datetime']
                        dt = datetime.fromisoformat(datetime_value.replace("Z", "+00:00")) 
                        readable_time = dt.strftime("%B %d, %Y %I:%M %p")
                        all_time.append(readable_time)

                    # Find all <div> elements with aria-label for likes, comments, and retweets
                    div_elements = soup.find_all('div', attrs={"aria-label": True})

                    for div_element in div_elements:
                        aria_label_value = div_element['aria-label']
                        try:
                            # if 'likes' in aria_label_value or ('bookmarks' in aria_label_value and 'views' in aria_label_value): 
                            #changed from or to and
                            if 'likes' in aria_label_value and 'replies' in aria_label_value and 'reposts' in aria_label_value and 'views' in aria_label_value: 
                                values = aria_label_value.split(', ')

                                try:
                                    replies = values[0].split(' ')[0]
                                    all_comments.append(replies)
                                except IndexError:
                                    replies = 'N/A'
                                try:
                                    retweets = values[1].split(' ')[0]
                                    all_retweets.append(retweets)
                                except IndexError:
                                    retweets = 'N/A'
                                try:
                                    likes = values[2].split(' ')[0]
                                    all_likes.append(likes)
                                except IndexError:
                                    likes = 'N/A'

                                for v in values:
                                    if 'views' in v:
                                        views = v.split(' ')[0]
                                        all_views.append(views)
                                        # print(views)

                        except: 
                            print("Failed to find aria labels!")

                    print(f'Time len {len(all_time)}')
                    print(f'Tweet len {len(tweet_texts)}')
                    print(f'Likes len {len(all_likes)}')
                    print(f'Retweets len {len(all_retweets)}')
                    print(f'Comments len {len(all_comments)}')
                    print(f'Views len {len(all_views)}')
                    
                    # Write to CSV
                    for time, tweet, likes, retweets, comments, views in zip(all_time, tweet_texts, all_likes, all_retweets, all_comments, all_views):
                        line = [name, time, tweet, likes, retweets, comments, views]
            
                        if tweet not in all_tweets and contains_keyword(tweet):
                        # if tweet not in all_tweets:
                            #Check whether there is a duplicate and that the tweet actually contains intended keywords
                            writer.writerow(line)
                            all_tweets.append(tweet)
                            if name not in account_processed:
                                account_processed.append(name)
                        else:
                            discard+=1 
                            # print("Duplicated tweet found.")
                    print(f"Sucessfully processed user {name}")

            except Exception as e:
                print(f"Error processing user: {str(e)}")
                exc_type, exc_obj, exc_tb = sys.exc_info()
                fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
                print(exc_type, fname, exc_tb.tb_lineno)
                continue
            finally:
                # print(f'Current count: {count}')
                count+=1
        
        #Check for accounts with parsed tweets but discarded because of irrelevance
        for a in account_pre_process:
            if a not in account_processed and a not in account_for_retry:
                account_irrelevant.append(a)
        #Check that accounts needed to be re-run is not processed list
        account_for_retry_set = set(account_for_retry)
        account_processed = [a for a in account_processed if a not in account_for_retry_set]

        #Writing run statistics at the end of the csv file
        now = datetime.now() #Current time that the data is being processed
        readable_now = now.strftime("%B %d, %Y %I:%M %p")

        writer.writerow([])
        writer.writerow(["|RUN STATISTICS|"])
        writer.writerow([f"Time Processed: {readable_now}", f"Number of discards: {discard}", f"List of accounts processed: {account_processed}", f"List of accounts coming in: {account_pre_process}", f"List of accounts needed to be rerun: {account_for_retry}", f"List of accounts with parsed but only irrelevant tweets: {account_irrelevant}"])

except Exception as e:
    print(f"Error: {str(e)}")
    exc_type, exc_obj, exc_tb = sys.exc_info()
    fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
    print(exc_type, fname, exc_tb.tb_lineno)

print(f"Number of discard for this run (from duplication and failure to contain keywords): {discard}")

Processing user: ThomasEWoods
Time len 10
Tweet len 10
Likes len 4
Retweets len 4
Comments len 4
Views len 4
Sucessfully processed user ThomasEWoods
Processing user: ThomasEWoods
Time len 9
Tweet len 9
Likes len 3
Retweets len 3
Comments len 3
Views len 3
Sucessfully processed user ThomasEWoods
Processing user: RealSpikeCohen
Time len 13
Tweet len 13
Likes len 6
Retweets len 6
Comments len 6
Views len 6
Sucessfully processed user RealSpikeCohen
Processing user: RealSpikeCohen
Time len 4
Tweet len 4
Likes len 4
Retweets len 4
Comments len 4
Views len 4
Sucessfully processed user RealSpikeCohen
Processing user: ComicDaveSmith
Time len 8
Tweet len 8
Likes len 4
Retweets len 4
Comments len 4
Views len 4
Sucessfully processed user ComicDaveSmith
Processing user: ComicDaveSmith
Time len 6
Tweet len 6
Likes len 3
Retweets len 3
Comments len 3
Views len 3
Sucessfully processed user ComicDaveSmith
Retry button appeared: FreemansMind96
Processing user: HannahDCox
Time len 5
Tweet len 5
Likes len